In [82]:
# from Modules.LSH import semantic_query_lsh
# from Modules.LSH import LSH


# import numpy as np
# import os

In [83]:
# file_path = "./random_data.txt"
# read_data = np.loadtxt(file_path)
# plane_norms = LSH(read_data, 8)
# query=[read_data[0]]
# folder_name = "bucket_files"
# result = semantic_query_lsh(query, plane_norms,folder_name)

In [2]:
%load_ext autoreload
%autoreload 2


from utils import *
from Modules.LSH import*
from api import *
from evaluation import *
from worst_case_implementation import VecDBWorst


datafile_path="../DataBase/random_data_10000.txt"

In [3]:
data_file='../DataBase/data.txt'
Level_1_path='../DataBase/Level1'
Level_2_path='../DataBase/Level2'
Level_3_path='../DataBase/Level3'

Level_1_nbits=8
Level_2_nbits=3
Level_3_nbits=3

data_api = DataApi(data_file)
# data_api.generate_data_file(10000)

In [6]:

# Test LSH_index
# Read Data From File
read_data = data_api.get_first_k_records(10000)


# Layer(1)
level_1_in=read_data
# TODO: Save Planes to be used in query Search
level_1_planes=LSH_index(data=level_1_in, nbits=Level_1_nbits,index_path=Level_1_path)

In [7]:
# Layer(2)
# On Each Bucket Apply LSH

# List all files in the directory
files = os.listdir(Level_1_path)

# TODO: Save Planes to be used in query Search
level_2_planes={}

# Loop over the files
for file_name in files:
    file_path = os.path.join(Level_1_path, file_name)
    
    if os.path.isfile(file_path):
        # Read Data
        read_data_2 = np.loadtxt(file_path,dtype=int,ndmin=1)

        level_2_in=data_api.get_multiple_records_by_ids(read_data_2)
        # level_2_in = array_to_dictionary(values=vectors,keys=np.hstack(read_data_2))

        # # Apply LSH on this Bucket
        level_2_planes[file_name[:-4]]=LSH_index(data=level_2_in.values(), nbits=Level_2_nbits,index_path=Level_2_path+'/' + file_name[:-4])



In [8]:
# Layer(3)
# On Each Bucket Apply LSH

# List all files in the directory
folders = os.listdir(Level_2_path)

# TODO: Save Planes to be used in query Search
level_3_planes={}
# Loop over the folders
for folder_name in folders:
    folder_path = os.path.join(Level_2_path, folder_name)
    files = os.listdir(folder_path)
    # Loop over the files
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.isfile(file_path):
            # Read Data
            read_data_3 = np.loadtxt(file_path,dtype=int,ndmin=1)

            level_3_in=data_api.get_multiple_records_by_ids(read_data_3)

            # # Apply LSH on this Bucket
            level_3_planes[file_name[:-4]]=LSH_index(data=level_3_in.values(), nbits=Level_3_nbits,index_path=Level_3_path+'/'+folder_name+'/' + file_name[:-4])


In [9]:
# Query
query=data_api.get_record_by_id(5)[5]['embed']
# Layer (1)
bucket_1,index_result_1 = semantic_query_lsh(query=query,plane_norms=level_1_planes,index_path=Level_1_path)
print("level(1)",bucket_1)
print("level(1)",len(index_result_1))

# Layer(2)
bucket_2,index_result_2 = semantic_query_lsh(query=query,plane_norms=level_2_planes[bucket_1],index_path=Level_2_path+"/"+bucket_1)
print("level(2)",bucket_2)
print("level(2)",len(index_result_2))

# Layer(3)
bucket_3,index_result_3 = semantic_query_lsh(query=query,plane_norms=level_3_planes[bucket_2],index_path=Level_3_path+"/"+bucket_1+'/'+bucket_2)
print("level(3)",bucket_3)
print("level(3)",len(index_result_3))



count = sum(element in index_result_2 for element in index_result_3)
print(count)



level(1) 01111000
level(1) 826
level(2) 010
level(2) 762
level(3) 100
level(3) 22
22


In [14]:
# query=data_api.get_first_k_records(5)[5]['embed']
db = VecDBWorst()
# records_np = np.random.random((10000, 70))
records_np=np.array(data_api.get_first_k_records(10000))[:]['embed']
print(records_np)
# # records_dict = [{"id": i, "embed": list(row)} for i, row in enumerate(records_np)]
# # _len = len(records_np)
# db.insert_records(read_data)
# res = run_queries(query,db, records_np, 5, 10)
# print(eval(res))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices